In [12]:
import pandas as pd
import numpy as np

unknown_injury = 0
df = pd.DataFrame()
for year in range(1975,2019,1):
    print()
    print('YEAR',year)
    acc = pd.read_csv('Data/{}/Accident.csv'.format(year), encoding = "ISO-8859-1")
    acc.head()
    acc_keys = set(acc.keys())

    veh = pd.read_csv('Data/{}/Vehicle.csv'.format(year), encoding = "ISO-8859-1")
    veh.head()
    veh_keys = set(veh.keys())

    per = pd.read_csv('Data/{}/PERSON.csv'.format(year), encoding = "ISO-8859-1")
    per_keys = set(per.keys())
    
    per_acc = per_keys.intersection(acc_keys)
    per_veh = per_keys.intersection(veh_keys)
    veh_acc = veh_keys.intersection(acc_keys)
    
    whole = pd.merge(per, veh, on=list(per_veh), how='left')
    whole = pd.merge(whole, acc, on=list(per_acc), how='left')
    
    try:
        print('test',np.sum(whole['MAN_REST'] == 1))
    except:
        whole['MAN_REST'] = whole['REST_USE']
        print('test',np.sum(whole['MAN_REST'] == 1))

    try:
        print('test',np.sum(whole['VE_FORMS'] == 1))
    except:
        whole['VE_FORMS'] = whole['PVE_FORMS']
        print('test',np.sum(whole['MAN_REST'] == 1))

    try:
        print('test',np.sum(whole['IMPACT1'] == 1))
    except:
        whole['IMPACT1'] = whole['PIMPACT1']
        print('test',np.sum(whole['IMPACT1'] == 1))


    keep_lst = ['AGE','SEX']
 

    toddler = whole.copy()    
    toddler['Lap Belt'] = 1*((toddler['MAN_REST'] == 2) + (toddler['MAN_REST'] == 3))

    toddler['Shoulder Belt'] = 1*((toddler['MAN_REST'] == 1) + (toddler['MAN_REST'] == 3))
    toddler['Car Seat'] = 1*((toddler['MAN_REST'] == 4) + (toddler['MAN_REST']  >= 10))
    toddler = toddler[toddler['AGE'] >= 2]
    toddler = toddler[toddler['AGE'] <= 6]
    print('2-6:', toddler.shape[0])
    toddler = toddler[toddler['SEAT_POS'] >= 1]  # in car only
    print('In Car:', toddler.shape[0])
    A = toddler.shape[0]
    toddler = toddler[toddler['INJ_SEV'] <= 4]
    print('Known Injury Level:', toddler.shape[0])
    unknown_injury = unknown_injury + A - toddler.shape[0]
    toddler['Fatal'] = 1*(toddler['INJ_SEV']==4)
    keep_lst.append('Fatal')
    toddler['Serious Injury+'] = 1*(toddler['INJ_SEV']>=3)
    keep_lst.append('Serious Injury+')
    toddler['Light Injury+'] = 1*(toddler['INJ_SEV']>=2)
    keep_lst.append('Light Injury+')
    toddler['Minor Injury+'] = 1*(toddler['INJ_SEV']>=1)
    keep_lst.append('Minor Injury+')
    toddler['Ejected'] = 1*((toddler['EJECTION'] == 1) + (toddler['EJECTION'] == 2))
    keep_lst.append('Ejected')
    toddler['Thrown'] = 1*(toddler['EJECTION'] == 1)
    keep_lst.append('Thrown')
    toddler['Vehicles Involved (not parked)'] = toddler['VE_FORMS']
    keep_lst.append('Vehicles Involved (not parked)')
    
    # Get collision type...
    toddler['Front Col'] = 1*((toddler['IMPACT1']==11)+(toddler['IMPACT1']==12)+(toddler['IMPACT1']==1))
    keep_lst.append('Front Col')

    toddler['Rear Col'] = 1*((toddler['IMPACT1']==7)+(toddler['IMPACT1']==6)+(toddler['IMPACT1']==5))
    keep_lst.append('Rear Col')

    toddler['Side Col'] = 1*( (toddler['IMPACT1']==2)+(toddler['IMPACT1']==3)+(toddler['IMPACT1']==4) + (toddler['IMPACT1']==8)+(toddler['IMPACT1']==9)+(toddler['IMPACT1']==10) + ((toddler['IMPACT1']>=61) & (toddler['IMPACT1']<=99)))
    keep_lst.append('Side Col')

    toddler['SCH_BUS'] = toddler['SCH_BUS'].fillna('')
    toddler = toddler[toddler['SPEC_USE']==0]  # just cars!
    
#     toddler['Striking'] = 1*((toddler['IMPACTS']==1) + (toddler['IMPACTS']==3))
#     toddler['Struck'] = 1*((toddler['IMPACTS']==2) + (toddler['IMPACTS']==3))
#     keep_lst.append('Striking')
#     keep_lst.append('Struck')
    
    toddler['Fire'] = 1*(toddler['FIRE_EXP']!=0)
    keep_lst.append('Fire')
    
    toddler = toddler[toddler['PER_TYP']==2]  # toddlers should not be driving
    
    
    toddler['Front Seat'] = 1*((toddler['SEAT_POS']>=1) & (toddler['SEAT_POS']<=3))
    keep_lst.append('Front Seat')
    try:
        toddler['Air Bag'] = 1*((toddler['AIR_BAG']==2)+(toddler['AIR_BAG']==3)+(toddler['AIR_BAG']==7)+(toddler['AIR_BAG']==8))
    except:
        toddler['Air Bag'] = 0
    
    try:
        toddler['SP_LIMIT'].head()
    except:
        toddler['SP_LIMIT'] = toddler['VSPD_LIM']
    keep_lst.append('SP_LIMIT')
    keep_lst.append('Air Bag')
    
    # do it
    toddler = toddler[keep_lst]
    df = pd.concat([df,toddler])    
    print(df.shape)
df.head()


YEAR 1975
test 102
test 48620
test 6628
2-6: 3798
In Car: 2731
Known Injury Level: 2720
(2707, 16)

YEAR 1976
test 75
test 49364
test 6360
2-6: 3621
In Car: 2626
Known Injury Level: 2623
(5310, 16)

YEAR 1977
test 103
test 50111
test 6840
2-6: 3562
In Car: 2653
Known Injury Level: 2647
(7942, 16)

YEAR 1978
test 103
test 50111
test 6840
2-6: 3562
In Car: 2653
Known Injury Level: 2647
(10574, 16)

YEAR 1979
test 124
test 51210
test 8901
2-6: 3279
In Car: 2441
Known Injury Level: 2434
(12987, 16)

YEAR 1980
test 134
test 53141
test 7606
2-6: 2925
In Car: 2180
Known Injury Level: 2163
(15141, 16)

YEAR 1981
test 114
test 50559
test 8561
2-6: 3043
In Car: 2371
Known Injury Level: 2359
(17492, 16)

YEAR 1982
test 118
test 47652
test 7853
2-6: 2877
In Car: 2262
Known Injury Level: 2245
(19663, 16)

YEAR 1983
test 140
test 45771
test 7512
2-6: 2957
In Car: 2351
Known Injury Level: 2340
(21909, 16)

YEAR 1984
test 216
test 46819
test 8290
2-6: 2954
In Car: 2381
Known Injury Level: 2369
(24173

,AGE,SEX,Fatal,Serious Injury+,Light Injury+,Minor Injury+,Ejected,Thrown,Vehicles Involved (not parked),Front Col,Rear Col,Side Col,Fire,Front Seat,SP_LIMIT,Air Bag
53,4,2,0,1,1,1,0,0,3,1,0,0,0,1,55.0,0
61,4,1,1,1,1,1,0,0,1,0,0,0,0,1,55.0,0
141,4,1,0,1,1,1,0,0,2,1,0,0,0,1,55.0,0
142,2,1,0,1,1,1,0,0,2,1,0,0,0,1,55.0,0
143,4,2,0,1,1,1,0,0,2,1,0,0,0,0,55.0,0


In [13]:
unknown_injury

705

In [14]:
df.to_csv('Data/toddler.csv', index=False)
df.to_csv('toddler.csv', index=False)